#STORED in codingbasu@gmail.com



In [1]:
from google.colab import drive
drive.mount('/content/gdrive') 
#data stored in basuarkaprabha1012, IN SEPERATED FORMAT

Mounted at /content/gdrive


In [2]:
#!unzip /content/gdrive/MyDrive/Bone_Removal_Dataset.zip
!unzip /content/gdrive/MyDrive/BRSeperated.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/Output/test/Normal/BRNormal-4255.png  
  inflating: content/Output/test/Normal/BRNormal-5486.png  
  inflating: content/Output/test/Normal/BRNormal-8473.png  
  inflating: content/Output/test/Normal/BRNormal-7274.png  
  inflating: content/Output/test/Normal/BRNormal-1275.png  
  inflating: content/Output/test/Normal/BRNormal-4644.png  
  inflating: content/Output/test/Normal/BRNormal-4971.png  
  inflating: content/Output/test/Normal/BRNormal-4682.png  
  inflating: content/Output/test/Normal/BRNormal-4788.png  
  inflating: content/Output/test/Normal/BRNormal-7432.png  
  inflating: content/Output/test/Normal/BRNormal-8598.png  
  inflating: content/Output/test/Normal/BRNormal-2601.png  
  inflating: content/Output/test/Normal/BRNormal-9438.png  
  inflating: content/Output/test/Normal/BRNormal-9265.png  
  inflating: content/Output/test/Normal/BRNormal-3431.png  
  inflating: content/Output/test/Normal/BRNormal-

In [3]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
pip install split-folders tqdm

In [5]:
import splitfolders  # or import split_folders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.

splitfolders.ratio("/content/Bone_Removal_Dataset", output="Output", seed=1337, ratio=(.6,.2,.2), group_prefix=None) # default values
# Split val/test with a fixed number of items e.g. 100 for each set.
# To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
#splitfolders.fixed("/content/susmitadipluskaggle", output="Output", seed=1337, fixed=(70,70), oversample=False, group_prefix=None) # default values

Copying files: 27790 files [00:06, 4059.75 files/s]


In [11]:
!zip -r /content/gdrive/MyDrive/BRSeperated.zip /content/Output

Streaming output truncated to the last 5000 lines.
  adding: content/Output/test/Normal/BRNormal-4255.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-5486.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-8473.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-7274.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-1275.png (deflated 1%)
  adding: content/Output/test/Normal/BRNormal-4644.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-4971.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-4682.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-4788.png (deflated 1%)
  adding: content/Output/test/Normal/BRNormal-7432.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-8598.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-2601.png (deflated 0%)
  adding: content/Output/test/Normal/BRNormal-9438.png (deflated 1%)
  adding: content/Output/test/Normal/BRNormal-9265.p

In [9]:
img_height,img_width=(228,228)
batch_size=64
train_data_dir="/content/content/Output/train"
valid_data_dir="/content/content/Output/val"
test_data_dir="/content/content/Output/test"

In [10]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                      train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                               …                                          shuffle='False')
valid_generator=train_datagen.flow_from_directory(valid_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle='False')
print(valid_generator.class_indices)
print(train_generator.class_indices)                            shuffle='False')
valid_generator=train_datagen.flow_from_directory(valid_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle='False')
print(valid_generator.class_indices)
print(train_generator.class_indices)

Found 16673 images belonging to 4 classes.
Found 5556 images belonging to 4 classes.
{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Pneumonia': 3}
{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Pneumonia': 3}


In [12]:
from keras.models import load_model
model=load_model('/content/gdrive/MyDrive/BR30(92.696).h5')
#stored in basuarkaprabha1012

In [17]:
model.compile(
  loss='categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [13]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
callbacks_list=[checkpoint]

In [25]:
!nvidia-smi

Sun May 16 09:31:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    41W /  70W |   9024MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
r = model.fit_generator(
  train_generator,
  epochs=20,
  steps_per_epoch=16673//(batch_size),
  validation_data=valid_generator,
  validation_steps=5556//(batch_size),
  callbacks=callbacks_list
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
260/260 [==============================] - 279s 1s/step - loss: 0.0905 - accuracy: 0.9683 - val_loss: 0.2493 - val_accuracy: 0.9326

Epoch 00001: val_accuracy did not improve from 0.93695
Epoch 2/20
260/260 [==============================] - 277s 1s/step - loss: 0.0961 - accuracy: 0.9672 - val_loss: 0.2568 - val_accuracy: 0.9351

Epoch 00002: val_accuracy did not improve from 0.93695
Epoch 3/20
260/260 [==============================] - 281s 1s/step - loss: 0.1001 - accuracy: 0.9647 - val_loss: 0.2618 - val_accuracy: 0.9337

Epoch 00003: val_accuracy did not improve from 0.93695
Epoch 4/20
260/260 [==============================] - 280s 1s/step - loss: 0.0876 - accuracy: 0.9711 - val_loss: 0.2743 - val_accuracy: 0.9248

Epoch 00004: val_accuracy did not improve from 0.93695
Epoch 5/20
260/260 [==============================] - 280s 1s/step - loss: 0.0960 - accuracy: 0.9662 - val_loss: 0.2730 - val_accuracy: 0.9281

Epoch 00005: val_accuracy did not improve from 0.93695
Epoch

In [49]:
test_generator=train_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle='False'
                                                  )
print(test_generator.class_indices)

Found 5561 images belonging to 4 classes.
{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Pneumonia': 3}


In [39]:
#model.save("BR70.h5")
from keras.models import load_model
model=load_model('/content/gdrive/MyDrive/BR70.h5')
#stored in basuarkaprabha1012

In [55]:
test_loss,test_acc=model.evaluate(test_generator,verbose=1)
print("Test accuracy:",test_acc)
print("Test Loss:",test_loss)

87/87 [==============================] - 70s 806ms/step - loss: 0.2647 - accuracy: 0.9283
Test accuracy: 0.9282503128051758
Test Loss: 0.26471608877182007


In [52]:
test_generator.shuffle=False
test_generator.reset()
Y_pred = model.predict_generator(test_generator,verbose=1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Covid','Lung_Opacity','Normal','Pneumonia']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))
cm=confusion_matrix(test_generator.classes, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print("class specific accuracy:")
print(cm.diagonal())
acsa = (cm[0,0]/sum(cm[0,:])+cm[1,1]/sum(cm[1,:])+cm[2,2]/sum(cm[2,:]))/3
print("average class specific accuracy :",acsa)
class_acc = [cm[i,i]/np.sum(cm[i,:]) if np.sum(cm[i,:]) else 0 for i in range(len(cm))]
print("class accuracy :",class_acc)
ppvs = [cm[i,i]/np.sum(cm[:,i]) if np.sum(cm[:,i]) else 0 for i in range(len(cm))]
print("predicted positive value :",ppvs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


87/87 [==============================] - 69s 793ms/step
Confusion Matrix
[[ 671   19   40  109]
 [  14 1128   59    2]
 [  31   95 1908    5]
 [   2    4    5 1469]]
Classification Report
              precision    recall  f1-score   support

       Covid       0.93      0.80      0.86       839
Lung_Opacity       0.91      0.94      0.92      1203
      Normal       0.95      0.94      0.94      2039
   Pneumonia       0.93      0.99      0.96      1480

    accuracy                           0.93      5561
   macro avg       0.93      0.92      0.92      5561
weighted avg       0.93      0.93      0.93      5561

class specific accuracy:
[0.79976162 0.93765586 0.93575282 0.99256757]
average class specific accuracy : 0.8910567671120967
class accuracy : [0.799761620977354, 0.9376558603491272, 0.9357528200098088, 0.9925675675675676]
predicted positive value : [0.965949252453617, 0.9287436027375324, 0.9033661852054455, 0.8810301796215363]


In [53]:
model.save("BR80.h5")

In [54]:
model.save("/content/gdrive/MyDrive/BR80.h5")